# Я ебал нормальное распределение, пошло оно в пиздень

# Распределение Релэя

In [91]:
import numpy as np
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm import tqdm

# CDF: $1 - e^{-\frac{x^2}{2\sigma^2}}$

In [4]:
np.random.rayleigh(2, 5)

array([5.68578342, 1.2304852 , 4.03550726, 2.26059891, 4.07225986])

In [75]:
del x

In [165]:
n,m = 12,1000
m_test = 200

# Train
X_train = np.random.rayleigh(np.random.rand(1)*100, n)
X_train.sort()
X_train = [X_train]
for _ in range(m-1):
    x = np.random.rayleigh(np.random.rand(1)*100, n)
    x.sort()
    x = [x]
    X_train = np.append(X_train, x, axis=0)
X_train = torch.Tensor(X_train)    

y_train = torch.Tensor([[_/n for _ in range(1,(n+1))] for j in range(m)])

# Test
X_test = np.random.rayleigh(np.random.rand(1)*100, n)
X_test.sort()
X_test = [X_test]
for _ in range(m_test-1):
    x = np.random.rayleigh(np.random.rand(1)*100, n)
    x.sort()
    x = [x]
    X_test = np.append(X_test, x, axis=0)
X_test = torch.Tensor(X_test)
    
y_test = torch.Tensor([[_/n for _ in range(1,(n+1))] for j in range(m_test)])

In [181]:
X_test[0]

tensor([  7.4706,  48.6477,  56.1582,  77.2318,  78.2408,  79.0046,  81.5424,
        122.3908, 130.0199, 143.5768, 187.8070, 206.9867])

In [182]:
y_test[0]

tensor([0.0833, 0.1667, 0.2500, 0.3333, 0.4167, 0.5000, 0.5833, 0.6667, 0.7500,
        0.8333, 0.9167, 1.0000])

In [183]:
class NormalNet(torch.nn.Module):
    def __init__(self, n_input, n_layer):
        super(NormalNet, self).__init__()
        self.fc1 = torch.nn.Linear(n_input, n_layer)
        self.act1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(n_layer, 1)
        
    def forward(self, x0):
        x = self.fc1(x0)
        x = self.act1(x)
        x = self.fc2(x)
        
        return (1 - (-x0**2/(2*x**2)).exp())

normal_net = NormalNet(n, 500)

def loss(pred, target):
    squares = (pred - target) ** 2
    return squares.sum()

optimizer = torch.optim.Adam(normal_net.parameters(), lr=0.01)

In [184]:
print('Start loss:', 1-((normal_net.forward(X_test) - y_test)**2).sum() / ((y_test - y_test.mean())**2).sum())

for epoch_index in range(10000):
    optimizer.zero_grad()

    y_pred = normal_net.forward(X_train)
    loss_val = loss(y_pred, y_train)
    
    if epoch_index % 500 == 0:
        print('Epoch %i' % epoch_index, 1-((normal_net.forward(X_test) - y_test)**2).sum() / ((y_test - y_test.mean())**2).sum())

    loss_val.backward()

    optimizer.step()

print('Finish loss:', 1-((normal_net.forward(X_test) - y_test)**2).sum() / ((y_test - y_test.mean())**2).sum())

Start loss: tensor(-2.3437, grad_fn=<RsubBackward1>)
Epoch 0 tensor(-2.3437, grad_fn=<RsubBackward1>)
Epoch 500 tensor(0.9338, grad_fn=<RsubBackward1>)
Epoch 1000 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 1500 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 2000 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 2500 tensor(0.9335, grad_fn=<RsubBackward1>)
Epoch 3000 tensor(0.9330, grad_fn=<RsubBackward1>)
Epoch 3500 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 4000 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 4500 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 5000 tensor(0.9335, grad_fn=<RsubBackward1>)
Epoch 5500 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 6000 tensor(0.9339, grad_fn=<RsubBackward1>)
Epoch 6500 tensor(0.9336, grad_fn=<RsubBackward1>)
Epoch 7000 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 7500 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 8000 tensor(0.9340, grad_fn=<RsubBackward1>)
Epoch 8500 tensor(0.9341, grad_fn=<RsubBackward1>)
Epoch 9000 tensor(0.9336, grad_f

In [180]:
1-((normal_net.forward(X_train) - y_train)**2).sum() / ((y_train - y_train.mean())**2).sum()

tensor(0.9335, grad_fn=<RsubBackward1>)

In [170]:
((normal_net.forward(X_test) - y_test)**2).sum()

tensor(13.1632, grad_fn=<SumBackward0>)